##### Needed data cleaning
* convert mis-categorized clockstops with defender named "Spikes"
* Pull in other dataset for coaches, offensive coordinators and coaches
* Make sure no null RushDirection on any PlayType == RUSH
* Make sure no null PassType on any PlayType == RUSH

##### predictions
* rush vs pass
* Will team convert 1st down? Series1stDown column
* punt, FG, go for it on 4th down
* Percentage predictions for e.g. Sack, Interception, Touchdown, Safety

##### Add columns
* Offensive score diff (how far ahead/behind is the team on offense)
* Game time remaining
* Half time remaining
* Coach
* Quarterback
* Offensive coordinator

In [243]:
import pandas as pd
import re

YEARS = range(2013, 2016)
CSV_NAMES = ['data/pbp-{}.csv'.format(year) for year in YEARS]

DROPNA_COLS = [
    'IsIncomplete',
    'YardLineDirection',
    'PlayType',
]

DROP_PLAYTYPES = {
    'KICK OFF',
    'EXCEPTION',
    'PENALTY',
    'TWO-POINT CONVERSION',
    'EXTRA POINT',
    'NO PLAY',
    'TIMEOUT',
    'FUMBLES',
}

DROP_COLUMNS = [
    'Challenger',
    'IsChallenge',
    'IsChallengeReversed',
    'IsFumble',
    'IsMeasurement',
    'IsNoPlay',
    'IsPenalty',
    'IsPenaltyAccepted',
    'IsTwoPointConversion',
    'IsTwoPointConversionSuccessful',
    'NextScore',
    'PenaltyTeam',
    'PenaltyType',
    'PenaltyYards',
    'TeamWin',
    'Unnamed: 10',
    'Unnamed: 12',
    'Unnamed: 16',
    'Unnamed: 17',
#     'YardLine',
#     'YardLineDirection',
#     'YardLineFixed',
#     'Yards',
]

# Using (?:...) operator to parse regex groups but not capture them.
DROP_DESC_REGEX = re.compile(r'^(?:END (?:GAME|QUARTER \d))|(?:TIMEOUT AT \d\d:\d\d.)|(?:TWO-MINUTE WARNING)$')

# Might change this to RUSH... depends
SCRAMBLE_REPLACE = 'PASS'

In [244]:

def clean_nfl_plays_dataframe(df):
    """Clean dataframe in a number of ways."""
    # Drop irrelevant columns
    df.drop(DROP_COLUMNS, axis=1, inplace=True)
    
    # Drop plays where both Description and PlayType are null -- unable to do anything with them
    df.drop(df[(df.Description.isnull() & df.PlayType.isnull())].index, inplace=True)

    # Drop plays which match description of various irrelevant events
    df.drop(df[df.Description.str.contains(DROP_DESC_REGEX)].index, inplace=True)

    # Convert PlayType of "direct snap" described plays to RUSH. Also fills many null PlayType values
    df.loc[df.Description.str.contains('DIRECT SNAP'), 'PlayType'] = 'RUSH'

    # Drop any remaining rows with null values in specific columns (weird data)
    df.dropna(subset=DROPNA_COLS, inplace=True)

    # Drop plays with irrelevant playtypes
    df.drop(df[df.PlayType.isin(DROP_PLAYTYPES)].index, inplace=True)
    
    # Drop abberation 0th down plays
    df.drop(df[df.Down == 0].index, inplace=True)
    
    # Change SACK PlayType to PASS -- assuming sack was made during pass attempt
    df.PlayType.replace('SACK', 'PASS', inplace=True)
    
    # Change SCRAMBLE PlayType to PASS -- assuming scramble was made during pass attempt
    df.PlayType.replace('SCRAMBLE', SCRAMBLE_REPLACE, inplace=True)
    
    return df


In [245]:
# Create the dataset
# Concatenate one dataset from all years except for last year which will be unseen data
uncleaned_plays = pd.concat([pd.read_csv(csv_name, header=0, error_bad_lines=False) for csv_name in CSV_NAMES[:-1]])

Skipping line 2331: expected 45 fields, saw 48
Skipping line 12467: expected 45 fields, saw 46
Skipping line 15362: expected 45 fields, saw 48
Skipping line 15376: expected 45 fields, saw 48

Skipping line 36878: expected 45 fields, saw 48
Skipping line 36885: expected 45 fields, saw 48
Skipping line 42706: expected 45 fields, saw 46



In [246]:
# Clean the dataset
plays = clean_nfl_plays_dataframe(uncleaned_plays)

GameId                   0
GameDate                 0
Quarter                  0
Minute                   0
Second                   0
OffenseTeam              0
DefenseTeam              0
Down                     0
ToGo                     0
YardLine                 0
SeriesFirstDown          0
Description              0
TeamWin                  0
SeasonYear               0
Yards                    0
Formation                0
PlayType                 0
IsRush                   0
IsPass                   0
IsIncomplete             0
IsTouchdown              0
PassType             31978
IsSack                   0
IsInterception           0
IsFumble                 0
RushDirection        42374
YardLineFixed            0
YardLineDirection        0
dtype: int64

In [250]:
tds = plays[plays.IsTouchdown == True]

In [251]:
tds.Description.values

array(['(5:45) 38-B.BOLDEN UP THE MIDDLE FOR 1 YARD, TOUCHDOWN.',
       '(7:17) 5-J.FLACCO PASS SHORT RIGHT TO 44-V.LEACH FOR 2 YARDS, TOUCHDOWN.',
       '(11:40) (SHOTGUN) 18-P.MANNING PASS DEEP MIDDLE TO 80-J.THOMAS FOR 24 YARDS, TOUCHDOWN [99-C.CANTY].',
       ...,
       '(5:38) (SHOTGUN) 8-M.SCHAUB PASS SHORT LEFT INTENDED FOR 85-K.THOMPKINS INTERCEPTED BY 22-T.JOHNSON AT OAK 43. 22-T.JOHNSON FOR 43 YARDS, TOUCHDOWN.',
       '(8:28) (NO HUDDLE) 10-E.MANNING PASS SHORT RIGHT TO 13-O.BECKHAM FOR 9 YARDS, TOUCHDOWN. PENALTY ON NYG-13-O.BECKHAM, TAUNTING, 15 YARDS, ENFORCED BETWEEN DOWNS.',
       '(4:55) 43-O.DARKWA LEFT TACKLE FOR 12 YARDS, TOUCHDOWN.'], dtype=object)